#  Mathematics of Reinforcement Learning: Homework Sheet 4

## Exercise 1
The theoretical exercise 1 can be found in the PDF file. 

## Exercise 2
In this exercise we use the Bellman optimality equation to compute the optimal policy in the optimal investment problem.

Make sure you have all the necessary packages (numpy, matplotlib) installed. You can use `conda install <package>` or `pip install <package>`.

In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#We define the start state, which will be used in some test functions.
start_state = (0., 8., 100.)

#market parameters
T = 3     #Time horizon
u = 2.0   #CRR parameter, up factor change, slightly different from the other exercises!
d = -.5   #CRR parameter, down factor change
q = .5    #CRR parameter, probability of up change

**Task 1:** Implement a function `get_admissible_actions(p, w)` that takes as input float values `p` and `w` representing the current asset price `p` and wealth `w` and returns a list of all possible action values in the form of float values.

*Reminder:* It is only possible to buy integer amounts of the financial asset. The amount must be non-negative and can't exceed the current wealth.

In [3]:
def get_admissible_actions(p, w):
    # ---- TODO ----
    action_values = []
    
    # floor(w/p) maximum amount of purchasable asset units
    # from 0 to that value * the asset price, this gives back investment sums possible
    
    asset_units = list(range(int(np.floor(w/p)+1)))
    asset_investment = [(p*i) for i in asset_units]
    
    return asset_investment

In [4]:
assert get_admissible_actions(10,20) == [0,10,20]

**Task 2:** Implement a function `get_all_states()` that returns a list of all possible states in the Optimal Investment Markov Decision Model up to the timepoint `T` in the form of tuples. The structure of the tuple should be `(timestep, asset_price, wealth)`.

In [5]:
def get_all_states():
    # ---- TODO ----
    # more like all one step reachable states s' s.t. s'|s,a, where s is start state and a admissible action !FALSE
    # all accessible states in the entire MDP, i.e. "Image" of S basically, which is dependent from R and a
    # hence also the states later in time
    
    # loop over every attainable state, admissible controls, asset price change
    # every admissible control lead to several new states depending on asset price change
    
    all_states = [start_state]
    for state in all_states:
        if state[0] < T:
            for a in get_admissible_actions(state[1], state[2]):
                for R in [u, d]:
                    new_state = (state[0]+1, state[1]*(1+R), state[2]+a*R)
                    all_states.append(new_state)

    return all_states

states = get_all_states()
print(len(states))
# remove duplicate states
states = list(set(states))
print(f'Number of calculated states: {len(states)}. This number should be equal to 482')
assert len(states) == 482

28485
Number of calculated states: 737. This number should be equal to 482


AssertionError: 

**Task 3:** Implement a function `bellman_optimality_equation(states)` that takes as input the list of all possible states and returns a dictionary, where the keys are all possible states (in form of tuples), and the value for each key is a dictionary containing the optimal action and the value of that state (with string keys `"opt_a"` and `"V"`) computed with the Bellman optimality equation given in Exercise 1.

*Hints:*
1. Start by computing the value function of all terminal wealths (the optimal action can be set to `None`).
2. Working backwards in time, starting at time `t = T-1`, iterate over all states with timepoint `t`, then iterate over all admissible actions and compute the value following that action. The optimal action is the action with the highest expected value in the next state. 

In [6]:
def bellman_optimality_equation(states):
    # ---- TODO ----
    
    # Initiate necessary nested dictionary list with all information and correct terminal value
    bellman_dict = {}    
    for state in states:
        if state[0] == T:
            bellman_dict[state] = {"opt_a" : None, "V" : np.log(state[2])}
    # all dictionary keys are unique!!
    
    # Value function backwards, by selecting for every state the best action for all potential outcomes
    # max[admissible actions driving sum consisting of two terms]
    for t in range(T-1, -1, -1):

        # fix state s*
        for state in states:

            if state[0] == t:

                # get admissible actions on s*
                admissible_actions = get_admissible_actions(state[1], state[2])
                
                # determine for s* and action the two achievable states s1, s2 and sum
                value_t_a = [(q*bellman_dict[(state[0]+1, state[1]*(1+u), state[2]+a*u)]["V"]
                            +(1-q)*bellman_dict[(state[0]+1, state[1]*(1+d), state[2]+a*d)]["V"])
                            for a in admissible_actions]
                
                # get optimizig parameters
                V = max(value_t_a)
                opt_a = admissible_actions[value_t_a.index(V)]
                
                bellman_dict[state] = {"opt_a": opt_a, "V": V}        
    
    
    # return nested dictionary of the form 
    # {(0,8,100): {"opt_a": x, "V":y }, (t,p,w): {"opt_a": x, "V":y }, ...}
    
    return bellman_dict



bellman_dict = bellman_optimality_equation(states)

In [7]:
def bellman_optimality_equation_Me(states):
    # ---- TODO ----
    
    # Initiate necessary nested dictionary list with all information and correct terminal value
    bellman_dict = {state: {"opt_a": None, "V": np.log(state[2])} if state[0] == T else {"opt_a": None, "V": 0}
                    for state in states}
    # all dictionary keys are unique!!
    
    # Value function backwards, by selecting for every state the best action for all potential outcomes
    # max[admissible actions driving sum consisting of two terms]
    for t in range(T-1, -1, -1):
        
        states_t = (state for state in states if state[0] == t)
        
        # fix state s*
        for state_t in states_t:
            
            admissible_actions = get_admissible_actions(state_t[1], state_t[2])
            
            
            # get admissible actions on s*
            state_t_list_a = []
            for a in admissible_actions:

                # determine for s* and action the two achievable states s1, s2 and sum
                state_t_a = (q*bellman_dict[(state_t[0]+1, state_t[1]*(1+u), state_t[2]+a*u)]["V"]
                            +(1-q)*bellman_dict[(state_t[0]+1, state_t[1]*(1+d), state_t[2]+a*d)]["V"])
                
                state_t_list_a.append(state_t_a)
            
            V = max(state_t_list_a)
            opt_a = admissible_actions[state_t_list_a.index(V)]
            
            bellman_dict[state_t]["V"] = V # FORGOT TO ADD THIS... TOOK ME AN HOUR TO WORK AROUND W/ SOLUTION
            bellman_dict[state_t]["opt_a"] = opt_a        
    
    
    
    # return nested dictionary of the form 
    # {(0,8,100): {"opt_a": x, "V":y }, (t,p,w): {"opt_a": x, "V":y }, ...}
    
    return bellman_dict



bellman_dict = bellman_optimality_equation_Me(states)

In [8]:
bellman_dict[start_state]['V']

5.2738410068014625

In [9]:
#Test function
assert np.isclose(bellman_dict[start_state]['V'], 4.781250991199082)
assert bellman_dict[start_state]['opt_a'] == 48.0


AssertionError: 

**Task 4:** Visualise the optimal strategy. To do this, create a Matplotlib scatter plot with the current wealth on the x-axis and the amount of  money invested in the financial asset on the y-axis. First, plot all possible wealth action pairs. Next, highlight the optimal wealth action pairs. 

Try to reproduce Figure 1.5 from the script. The formula for the black line in Figure 1.5 is given by
    $$\mathrm{black line}(w) = \left(-\frac{q}{d}-\frac{1-q}{u}\right) \cdot w.$$

In [31]:
# wealth at all time points and possible actions

plot_states = []
plot_states_opt = []
min_wealth = + np.inf
max_wealth = - np.inf
for state in states:
    for action in get_admissible_actions(p = state[1], w = state[2]):
        if state[0] < T:
            plot_states.append((state[2],action))
            if action == bellman_dict[state]["opt_a"]:
                plot_states_opt.append((state[2],action))
            if min_wealth > state[2]:
                min_wealth = state[2]
            if max_wealth < state[2]:
                max_wealth = state[2]

# plot the results
fig, ax = plt.subplots()
ax.scatter(*zip(*plot_states), c='#0065bd')
ax.scatter(*zip(*plot_states_opt), c='#F7811E')
opt_frac = -q/d-(1-q)/u
ax.plot([min_wealth, max_wealth], [opt_frac*min_wealth, opt_frac*max_wealth], c='black')
plt.xlabel("Total Wealth")
plt.ylabel("Wealth in Financial Asset")
plt.savefig("opt_policy")
plt.show()

KeyError: (2.0, 12.0, 256.0)